In [ ]:
# DEEP ANALYSIS: Advanced Feature Engineering & Interaction Discovery
# Professional Data Science Approach to Fertilizer Recommendation Dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Advanced libraries
from sklearn.preprocessing import StandardScaler, LabelEncoder, PolynomialFeatures
from sklearn.feature_selection import mutual_info_classif, SelectKBest, chi2
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from scipy import stats
from scipy.stats import chi2_contingency
import itertools

# Set advanced display options
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("🔬 DEEP ANALYSIS: FERTILIZER RECOMMENDATION DATASET")
print("=" * 60)
print("Loading data and preparing for advanced analysis...")

# Load datasets
data_dir = Path('datasets')
train_df = pd.read_csv(data_dir / 'train.csv')
test_df = pd.read_csv(data_dir / 'test.csv')

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")
print("✅ Data loaded successfully")


In [ ]:
# 🧬 DOMAIN-SPECIFIC FEATURE ENGINEERING
print("\n🧬 AGRICULTURAL DOMAIN FEATURE ENGINEERING")
print("=" * 50)

# Create a working copy
train_engineered = train_df.copy()
test_engineered = test_df.copy()

# 1. NPK RATIO FEATURES - Critical in Agriculture
print("\n1. NPK Ratio Features:")

def create_npk_features(df):
    """Create NPK-based agricultural features"""
    df = df.copy()
    
    # Basic NPK ratios
    df['N_P_ratio'] = df['Nitrogen'] / (df['Phosphorous'] + 1e-6)  # Avoid division by zero
    df['N_K_ratio'] = df['Nitrogen'] / (df['Potassium'] + 1e-6)
    df['P_K_ratio'] = df['Phosphorous'] / (df['Potassium'] + 1e-6)
    
    # NPK sum and mean - total nutrient load
    df['NPK_sum'] = df['Nitrogen'] + df['Phosphorous'] + df['Potassium']
    df['NPK_mean'] = df['NPK_sum'] / 3
    
    # NPK balance index (deviation from equal ratios)
    df['NPK_balance'] = df[['Nitrogen', 'Phosphorous', 'Potassium']].std(axis=1)
    
    # Dominant nutrient
    npk_cols = ['Nitrogen', 'Phosphorous', 'Potassium']
    df['dominant_nutrient'] = df[npk_cols].idxmax(axis=1)
    
    # NPK categorical ratios (High/Medium/Low)
    for nutrient in ['Nitrogen', 'Phosphorous', 'Potassium']:
        df[f'{nutrient}_level'] = pd.cut(df[nutrient], 
                                       bins=3, 
                                       labels=['Low', 'Medium', 'High'])
    
    return df

train_engineered = create_npk_features(train_engineered)
test_engineered = create_npk_features(test_engineered)

print("✅ NPK ratio features created:")
npk_features = ['N_P_ratio', 'N_K_ratio', 'P_K_ratio', 'NPK_sum', 'NPK_mean', 'NPK_balance']
for feat in npk_features:
    print(f"   {feat}: {train_engineered[feat].describe()['mean']:.3f} ± {train_engineered[feat].describe()['std']:.3f}")

# 2. ENVIRONMENTAL COMPOSITE FEATURES
print("\n2. Environmental Composite Features:")

def create_environmental_features(df):
    """Create environmental composite features"""
    df = df.copy()
    
    # Climate stress index
    df['temp_stress'] = np.where(df['Temparature'] > 35, 1, 0)  # High temperature stress
    df['humidity_stress'] = np.where(df['Humidity'] < 55, 1, 0)  # Low humidity stress
    df['moisture_stress'] = np.where(df['Moisture'] < 30, 1, 0)  # Low moisture stress
    df['environmental_stress'] = df['temp_stress'] + df['humidity_stress'] + df['moisture_stress']
    
    # Climate comfort index
    df['climate_comfort'] = (
        (df['Temparature'] - df['Temparature'].min()) / (df['Temparature'].max() - df['Temparature'].min()) +
        (df['Humidity'] - df['Humidity'].min()) / (df['Humidity'].max() - df['Humidity'].min()) +
        (df['Moisture'] - df['Moisture'].min()) / (df['Moisture'].max() - df['Moisture'].min())
    ) / 3
    
    # Evapotranspiration proxy (simplified)
    df['ET_proxy'] = df['Temparature'] * (100 - df['Humidity']) / 100
    
    # Water availability index
    df['water_availability'] = df['Humidity'] * df['Moisture'] / 100
    
    return df

train_engineered = create_environmental_features(train_engineered)
test_engineered = create_environmental_features(test_engineered)

print("✅ Environmental features created:")
env_features = ['environmental_stress', 'climate_comfort', 'ET_proxy', 'water_availability']
for feat in env_features:
    print(f"   {feat}: {train_engineered[feat].describe()['mean']:.3f} ± {train_engineered[feat].describe()['std']:.3f}")


In [ ]:
# 🔗 ADVANCED INTERACTION ANALYSIS
print("\n🔗 ADVANCED FEATURE INTERACTIONS")
print("=" * 50)

# 3. CROP-SOIL INTERACTIONS
print("\n3. Crop-Soil Domain Interactions:")

def create_crop_soil_interactions(df):
    """Create meaningful crop-soil interaction features"""
    df = df.copy()
    
    # Crop-Soil combination encoding
    df['crop_soil_combo'] = df['Crop Type'] + "_" + df['Soil Type']
    
    # Soil suitability for crop (domain knowledge proxy)
    soil_drainage = {'Sandy': 'high', 'Loamy': 'medium', 'Clayey': 'low', 'Red': 'medium', 'Black': 'low'}
    df['soil_drainage'] = df['Soil Type'].map(soil_drainage)
    
    # Water-demanding crops vs soil moisture retention
    water_demanding_crops = ['Paddy', 'Sugarcane']
    moisture_retaining_soils = ['Clayey', 'Black', 'Loamy']
    
    df['crop_water_demand'] = df['Crop Type'].isin(water_demanding_crops).astype(int)
    df['soil_water_retention'] = df['Soil Type'].isin(moisture_retaining_soils).astype(int)
    df['water_match'] = df['crop_water_demand'] * df['soil_water_retention']
    
    return df

train_engineered = create_crop_soil_interactions(train_engineered)
test_engineered = create_crop_soil_interactions(test_engineered)

print("✅ Crop-soil interactions created")
print(f"   Unique crop-soil combinations: {train_engineered['crop_soil_combo'].nunique()}")
print(f"   Water demand match distribution: {train_engineered['water_match'].value_counts().to_dict()}")

# 4. NPK-ENVIRONMENT INTERACTIONS
print("\n4. NPK-Environment Interactions:")

def create_npk_environment_interactions(df):
    """Create NPK-environment interaction features"""
    df = df.copy()
    
    # Nutrient efficiency under different conditions
    df['N_efficiency'] = df['Nitrogen'] * df['water_availability']  # N efficiency with water
    df['P_mobility'] = df['Phosphorous'] * (df['Temparature'] - 25) / 10  # P mobility with temp
    df['K_leaching_risk'] = df['Potassium'] * df['Moisture'] / df['Humidity']  # K leaching risk
    
    # Temperature-adjusted nutrient needs
    df['temp_adjusted_N'] = df['Nitrogen'] * (1 + (df['Temparature'] - 30) / 100)
    df['temp_adjusted_P'] = df['Phosphorous'] * (1 + (df['Temparature'] - 30) / 200)
    
    # Humidity-adjusted requirements
    df['humidity_adjusted_K'] = df['Potassium'] * (df['Humidity'] / 60)
    
    return df

train_engineered = create_npk_environment_interactions(train_engineered)
test_engineered = create_npk_environment_interactions(test_engineered)

print("✅ NPK-environment interactions created")

# 5. POLYNOMIAL & STATISTICAL INTERACTIONS
print("\n5. Statistical Interaction Features:")

def create_statistical_interactions(df):
    """Create statistical interaction features"""
    df = df.copy()
    
    # Numerical feature interactions (top combinations)
    df['temp_humidity'] = df['Temparature'] * df['Humidity']
    df['moisture_npk'] = df['Moisture'] * df['NPK_sum']
    df['temp_nitrogen'] = df['Temparature'] * df['Nitrogen']
    
    # Polynomial features for key nutrients
    df['nitrogen_squared'] = df['Nitrogen'] ** 2
    df['phosphorous_squared'] = df['Phosphorous'] ** 2
    df['potassium_squared'] = df['Potassium'] ** 2
    
    # Cross-ratios
    df['temp_moisture_ratio'] = df['Temparature'] / (df['Moisture'] + 1e-6)
    df['npk_env_ratio'] = df['NPK_sum'] / (df['climate_comfort'] + 1e-6)
    
    return df

train_engineered = create_statistical_interactions(train_engineered)
test_engineered = create_statistical_interactions(test_engineered)

print("✅ Statistical interactions created")
print(f"\nTotal engineered features: {train_engineered.shape[1]} (original: {train_df.shape[1]})")
print(f"New features added: {train_engineered.shape[1] - train_df.shape[1]}")


In [ ]:
# 📊 FEATURE IMPORTANCE & SELECTION ANALYSIS
print("\n📊 ADVANCED FEATURE IMPORTANCE ANALYSIS")
print("=" * 50)

# Prepare encoded data for analysis
def prepare_encoded_data(df, target_col=None):
    """Prepare data with proper encoding for ML analysis"""
    df_encoded = df.copy()
    
    # Label encode categorical features
    categorical_features = df_encoded.select_dtypes(include=['object']).columns
    le_dict = {}
    
    for col in categorical_features:
        if col != target_col:  # Don't encode target if provided
            le = LabelEncoder()
            df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
            le_dict[col] = le
    
    return df_encoded, le_dict

# Encode the data
train_encoded, label_encoders = prepare_encoded_data(train_engineered, 'Fertilizer Name')
target_encoded = LabelEncoder().fit_transform(train_engineered['Fertilizer Name'])

# Get numerical features only for analysis
numerical_features = train_encoded.select_dtypes(include=[np.number]).columns.tolist()
if 'id' in numerical_features:
    numerical_features.remove('id')

X_encoded = train_encoded[numerical_features]

print(f"Features for analysis: {len(numerical_features)}")

# 1. MUTUAL INFORMATION ANALYSIS
print("\n1. Comprehensive Mutual Information Analysis:")

mi_scores = mutual_info_classif(X_encoded, target_encoded, random_state=42)
mi_results = pd.DataFrame({
    'Feature': numerical_features,
    'Mutual_Information': mi_scores
}).sort_values('Mutual_Information', ascending=False)

print("\nTop 15 Features by Mutual Information:")
print(mi_results.head(15).to_string(index=False))

# Visualize top features
plt.figure(figsize=(12, 8))
top_features = mi_results.head(20)
sns.barplot(data=top_features, x='Mutual_Information', y='Feature')
plt.title('Top 20 Features by Mutual Information Score')
plt.xlabel('Mutual Information Score')
plt.tight_layout()
plt.show()

# 2. RANDOM FOREST FEATURE IMPORTANCE
print("\n2. Random Forest Feature Importance:")

rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_encoded, target_encoded)

rf_importance = pd.DataFrame({
    'Feature': numerical_features,
    'RF_Importance': rf.feature_importances_
}).sort_values('RF_Importance', ascending=False)

print("\nTop 15 Features by Random Forest Importance:")
print(rf_importance.head(15).to_string(index=False))


In [ ]:
# 🎯 FEATURE INTERACTION DEEP DIVE
print("\n🎯 FEATURE INTERACTION DEEP DIVE")
print("=" * 50)

# Combine importance scores
feature_comparison = mi_results.merge(rf_importance, on='Feature')
feature_comparison['combined_score'] = (
    feature_comparison['Mutual_Information'] * 0.6 + 
    feature_comparison['RF_Importance'] * 0.4
)
feature_comparison = feature_comparison.sort_values('combined_score', ascending=False)

print("\n3. Combined Feature Ranking (MI + RF):")
print(feature_comparison.head(15)[['Feature', 'combined_score']].to_string(index=False))

# 4. INTERACTION EFFECT ANALYSIS
print("\n4. Deep Interaction Effect Analysis:")

def analyze_feature_interactions(df, target, top_n=10):
    """Analyze pairwise feature interactions"""
    top_features = feature_comparison.head(top_n)['Feature'].tolist()
    interaction_strength = {}
    
    print(f"\nAnalyzing interactions between top {top_n} features...")
    
    for i, feat1 in enumerate(top_features):
        for j, feat2 in enumerate(top_features[i+1:], i+1):
            # Create interaction feature
            interaction_feat = df[feat1] * df[feat2]
            
            # Calculate mutual information of interaction
            mi_interaction = mutual_info_classif(
                interaction_feat.values.reshape(-1, 1), 
                target, 
                random_state=42
            )[0]
            
            # Calculate individual MI scores
            mi_feat1 = mutual_info_classif(
                df[feat1].values.reshape(-1, 1), 
                target, 
                random_state=42
            )[0]
            
            mi_feat2 = mutual_info_classif(
                df[feat2].values.reshape(-1, 1), 
                target, 
                random_state=42
            )[0]
            
            # Interaction strength = MI(interaction) - max(MI(feat1), MI(feat2))
            interaction_strength[f"{feat1}_x_{feat2}"] = {
                'interaction_mi': mi_interaction,
                'feat1_mi': mi_feat1,
                'feat2_mi': mi_feat2,
                'synergy': mi_interaction - max(mi_feat1, mi_feat2)
            }
    
    return interaction_strength

# Analyze interactions (limited to top features for computational efficiency)
interactions = analyze_feature_interactions(X_encoded, target_encoded, top_n=8)

# Sort by synergy effect
interaction_df = pd.DataFrame(interactions).T
interaction_df = interaction_df.sort_values('synergy', ascending=False)

print("\nTop 10 Feature Interactions by Synergy Effect:")
print(interaction_df.head(10)[['interaction_mi', 'synergy']].round(4))

# 5. ADVANCED CATEGORICAL ANALYSIS
print("\n5. Advanced Categorical Feature Analysis:")

# Analyze categorical combinations with target
categorical_importance = {}

# Original categorical features
for cat_feat in ['Crop Type', 'Soil Type']:
    # Calculate chi-square test
    contingency_table = pd.crosstab(train_engineered[cat_feat], train_engineered['Fertilizer Name'])
    chi2_stat, p_val, dof, expected = chi2_contingency(contingency_table)
    
    categorical_importance[cat_feat] = {
        'chi2_statistic': chi2_stat,
        'p_value': p_val,
        'cramers_v': np.sqrt(chi2_stat / (contingency_table.sum().sum() * (min(contingency_table.shape) - 1)))
    }
    
    print(f"\n{cat_feat}:")
    print(f"  Chi-square statistic: {chi2_stat:.2f}")
    print(f"  P-value: {p_val:.2e}")
    print(f"  Cramér's V: {categorical_importance[cat_feat]['cramers_v']:.4f}")

# Analyze engineered categorical combinations
print(f"\nCrop-Soil Combination Analysis:")
crop_soil_table = pd.crosstab(train_engineered['crop_soil_combo'], train_engineered['Fertilizer Name'])
chi2_combo, p_combo, _, _ = chi2_contingency(crop_soil_table)
cramers_v_combo = np.sqrt(chi2_combo / (crop_soil_table.sum().sum() * (min(crop_soil_table.shape) - 1)))

print(f"  Chi-square statistic: {chi2_combo:.2f}")
print(f"  P-value: {p_combo:.2e}")
print(f"  Cramér's V: {cramers_v_combo:.4f}")
print(f"  Unique combinations: {train_engineered['crop_soil_combo'].nunique()}")
print(f"  Most predictive combinations:")

# Show most predictive crop-soil combinations
combo_fert_strength = []
for combo in train_engineered['crop_soil_combo'].unique():
    combo_data = train_engineered[train_engineered['crop_soil_combo'] == combo]
    most_common_fert = combo_data['Fertilizer Name'].mode().iloc[0]
    fert_percentage = (combo_data['Fertilizer Name'] == most_common_fert).mean()
    combo_fert_strength.append({
        'combination': combo,
        'dominant_fertilizer': most_common_fert,
        'dominance': fert_percentage,
        'sample_size': len(combo_data)
    })

combo_strength_df = pd.DataFrame(combo_fert_strength)
combo_strength_df = combo_strength_df.sort_values('dominance', ascending=False)
print(combo_strength_df.head(10).to_string(index=False))


In [ ]:
# 🔍 CLUSTERING & PATTERN DISCOVERY
print("\n🔍 UNSUPERVISED PATTERN DISCOVERY")
print("=" * 50)

# 6. CLUSTERING ANALYSIS
print("\n6. K-Means Clustering Analysis:")

# Prepare data for clustering (scale numerical features)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)

# Determine optimal number of clusters using silhouette score
from sklearn.metrics import silhouette_score

silhouette_scores = []
K_range = range(2, 12)
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(X_scaled)
    silhouette_avg = silhouette_score(X_scaled, cluster_labels)
    silhouette_scores.append(silhouette_avg)

optimal_k = K_range[np.argmax(silhouette_scores)]
print(f"Optimal number of clusters: {optimal_k} (silhouette score: {max(silhouette_scores):.3f})")

# Perform clustering with optimal K
kmeans_optimal = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
cluster_labels = kmeans_optimal.fit_predict(X_scaled)

# Add cluster labels to dataframe
train_clustered = train_engineered.copy()
train_clustered['cluster'] = cluster_labels

# Analyze cluster characteristics
print(f"\nCluster Analysis:")
cluster_analysis = {}
for cluster_id in range(optimal_k):
    cluster_data = train_clustered[train_clustered['cluster'] == cluster_id]
    
    # Most common fertilizer in cluster
    most_common_fert = cluster_data['Fertilizer Name'].mode().iloc[0]
    fert_purity = (cluster_data['Fertilizer Name'] == most_common_fert).mean()
    
    # Average feature values
    avg_features = cluster_data[['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Phosphorous', 'Potassium']].mean()
    
    cluster_analysis[cluster_id] = {
        'size': len(cluster_data),
        'dominant_fertilizer': most_common_fert,
        'purity': fert_purity,
        'avg_temp': avg_features['Temparature'],
        'avg_nitrogen': avg_features['Nitrogen'],
        'avg_phosphorous': avg_features['Phosphorous'],
        'avg_potassium': avg_features['Potassium']
    }
    
    print(f"\nCluster {cluster_id}:")
    print(f"  Size: {len(cluster_data)} samples")
    print(f"  Dominant fertilizer: {most_common_fert} ({fert_purity:.1%} purity)")
    print(f"  Avg NPK: N={avg_features['Nitrogen']:.1f}, P={avg_features['Phosphorous']:.1f}, K={avg_features['Potassium']:.1f}")

# 7. PCA ANALYSIS
print("\n7. Principal Component Analysis:")

pca = PCA(n_components=0.95)  # Keep 95% of variance
X_pca = pca.fit_transform(X_scaled)

print(f"Original features: {X_scaled.shape[1]}")
print(f"PCA components for 95% variance: {X_pca.shape[1]}")
print(f"Explained variance by first 5 components: {pca.explained_variance_ratio_[:5]}")

# Plot PCA components
plt.figure(figsize=(15, 5))

# Explained variance
plt.subplot(1, 3, 1)
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), 
         np.cumsum(pca.explained_variance_ratio_), 'bo-')
plt.xlabel('Principal Component')
plt.ylabel('Cumulative Explained Variance')
plt.title('PCA Explained Variance')
plt.grid(True)

# Feature loadings for first two components
plt.subplot(1, 3, 2)
feature_loadings = pd.DataFrame(
    pca.components_[:2].T,
    columns=['PC1', 'PC2'],
    index=numerical_features
)
top_features_pc = feature_loadings.abs().sum(axis=1).nlargest(10).index
plt.scatter(feature_loadings.loc[top_features_pc, 'PC1'], 
           feature_loadings.loc[top_features_pc, 'PC2'])
for i, feature in enumerate(top_features_pc):
    plt.annotate(feature, 
                (feature_loadings.loc[feature, 'PC1'], 
                 feature_loadings.loc[feature, 'PC2']),
                fontsize=8, rotation=45)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Feature Loadings (Top 10)')

# 2D PCA plot colored by fertilizer
plt.subplot(1, 3, 3)
colors = plt.cm.Set3(np.linspace(0, 1, len(train_engineered['Fertilizer Name'].unique())))
for i, fertilizer in enumerate(train_engineered['Fertilizer Name'].unique()):
    mask = train_engineered['Fertilizer Name'] == fertilizer
    plt.scatter(X_pca[mask, 0], X_pca[mask, 1], 
               c=[colors[i]], label=fertilizer, alpha=0.6, s=1)
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.title('PCA: Fertilizer Separation')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)

plt.tight_layout()
plt.show()


In [ ]:
# 🧠 ADVANCED PATTERN MINING & INSIGHTS
print("\n🧠 ADVANCED PATTERN MINING")
print("=" * 50)

# 8. FERTILIZER DECISION RULES EXTRACTION
print("\n8. Decision Rule Extraction:")

from sklearn.tree import DecisionTreeClassifier, export_text

# Train a decision tree for rule extraction
dt = DecisionTreeClassifier(max_depth=5, min_samples_split=100, random_state=42)
dt.fit(X_encoded, target_encoded)

# Extract and display rules
tree_rules = export_text(dt, feature_names=numerical_features, 
                        class_names=LabelEncoder().fit(train_engineered['Fertilizer Name']).classes_,
                        max_depth=3)
print("Decision Tree Rules (Top 3 levels):")
print("=" * 40)
print(tree_rules[:2000] + "..." if len(tree_rules) > 2000 else tree_rules)

# 9. CORRELATION NETWORK ANALYSIS
print("\n9. Feature Correlation Network Analysis:")

# Calculate correlation matrix for engineered features
corr_matrix = X_encoded.corr()

# Find strong correlations (>0.7)
strong_correlations = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        corr_val = corr_matrix.iloc[i, j]
        if abs(corr_val) > 0.7:
            strong_correlations.append({
                'feature1': corr_matrix.columns[i],
                'feature2': corr_matrix.columns[j],
                'correlation': corr_val
            })

strong_corr_df = pd.DataFrame(strong_correlations)
if len(strong_corr_df) > 0:
    strong_corr_df = strong_corr_df.sort_values('correlation', key=abs, ascending=False)
    print(f"Strong correlations found (|r| > 0.7): {len(strong_corr_df)}")
    print(strong_corr_df.head(10))
else:
    print("No strong correlations found (|r| > 0.7)")

# 10. FEATURE EFFICIENCY ANALYSIS
print("\n10. Feature Engineering Efficiency Analysis:")

# Compare performance with original vs engineered features
original_features = ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous']
original_categorical = ['Crop Type', 'Soil Type']

# Encode original categorical features
original_data = train_df[original_features + original_categorical].copy()
for col in original_categorical:
    le = LabelEncoder()
    original_data[col] = le.fit_transform(original_data[col])

# Calculate baseline performance
baseline_mi = mutual_info_classif(original_data, target_encoded, random_state=42)
baseline_score = np.mean(baseline_mi)

# Calculate engineered feature performance
engineered_mi = mutual_info_classif(X_encoded, target_encoded, random_state=42)
engineered_score = np.mean(engineered_mi)

print(f"Baseline feature MI score: {baseline_score:.4f}")
print(f"Engineered feature MI score: {engineered_score:.4f}")
print(f"Improvement: {((engineered_score - baseline_score) / baseline_score * 100):+.1f}%")

# Feature value-add analysis
feature_value_add = pd.DataFrame({
    'Feature_Type': ['Original'] * len(original_features + original_categorical) + 
                   ['Engineered'] * (len(numerical_features) - len(original_features)),
    'Feature': original_features + original_categorical + 
              [f for f in numerical_features if f not in original_features],
    'MI_Score': list(baseline_mi) + 
               [mi_results[mi_results['Feature'] == f]['Mutual_Information'].iloc[0] 
                for f in numerical_features if f not in original_features]
})

# Top engineered features
top_engineered = feature_value_add[feature_value_add['Feature_Type'] == 'Engineered'].nlargest(10, 'MI_Score')
print(f"\nTop 10 Engineered Features by MI Score:")
print(top_engineered[['Feature', 'MI_Score']].to_string(index=False))


In [ ]:
# 🎨 ADVANCED VISUALIZATION & INSIGHTS
print("\n🎨 ADVANCED VISUALIZATION SUITE")
print("=" * 50)

# 11. COMPREHENSIVE FEATURE INTERACTION HEATMAP
print("\n11. Feature Interaction Visualization:")

# Create comprehensive interaction heatmap
fig, axes = plt.subplots(2, 2, figsize=(20, 16))

# Top features correlation heatmap
top_15_features = feature_comparison.head(15)['Feature'].tolist()
top_corr_matrix = X_encoded[top_15_features].corr()

sns.heatmap(top_corr_matrix, annot=True, cmap='RdBu_r', center=0, 
            fmt='.2f', ax=axes[0,0])
axes[0,0].set_title('Top 15 Features Correlation Matrix')

# NPK feature relationships
npk_related = [f for f in numerical_features if any(x in f.lower() for x in ['nitrogen', 'phosphorous', 'potassium', 'npk', 'n_p', 'n_k', 'p_k'])]
if len(npk_related) > 1:
    npk_corr = X_encoded[npk_related].corr()
    sns.heatmap(npk_corr, annot=True, cmap='viridis', center=0, 
                fmt='.2f', ax=axes[0,1])
    axes[0,1].set_title('NPK-Related Features Correlation')

# Environmental feature relationships
env_related = [f for f in numerical_features if any(x in f.lower() for x in ['temp', 'humidity', 'moisture', 'climate', 'stress', 'water'])]
if len(env_related) > 1:
    env_corr = X_encoded[env_related].corr()
    sns.heatmap(env_corr, annot=True, cmap='plasma', center=0, 
                fmt='.2f', ax=axes[1,0])
    axes[1,0].set_title('Environmental Features Correlation')

# Feature importance comparison
importance_comparison = pd.DataFrame({
    'Feature': mi_results['Feature'],
    'Mutual_Information': mi_results['Mutual_Information'],
    'RF_Importance': mi_results['Feature'].map(dict(zip(rf_importance['Feature'], rf_importance['RF_Importance'])))
})

axes[1,1].scatter(importance_comparison['Mutual_Information'], 
                 importance_comparison['RF_Importance'], alpha=0.6)
axes[1,1].set_xlabel('Mutual Information')
axes[1,1].set_ylabel('Random Forest Importance')
axes[1,1].set_title('Feature Importance Comparison')

# Add feature names for top features
top_features_viz = importance_comparison.nlargest(10, 'Mutual_Information')
for idx, row in top_features_viz.iterrows():
    axes[1,1].annotate(row['Feature'][:15], (row['Mutual_Information'], row['RF_Importance']),
                      fontsize=8, alpha=0.7)

plt.tight_layout()
plt.show()

# 12. FERTILIZER RECOMMENDATION PATTERNS VISUALIZATION
print("\n12. Fertilizer Recommendation Patterns:")

fig, axes = plt.subplots(2, 3, figsize=(20, 12))

# NPK distribution by fertilizer
fertilizers = train_engineered['Fertilizer Name'].unique()
colors = plt.cm.Set3(np.linspace(0, 1, len(fertilizers)))

for i, nutrient in enumerate(['Nitrogen', 'Phosphorous', 'Potassium']):
    ax = axes[0, i]
    for j, fert in enumerate(fertilizers):
        fert_data = train_engineered[train_engineered['Fertilizer Name'] == fert]
        ax.hist(fert_data[nutrient], alpha=0.6, label=fert, color=colors[j], bins=20)
    ax.set_xlabel(nutrient)
    ax.set_ylabel('Frequency')
    ax.set_title(f'{nutrient} Distribution by Fertilizer')
    if i == 2:  # Only show legend on last plot
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Environmental conditions by fertilizer
for i, condition in enumerate(['Temparature', 'Humidity', 'Moisture']):
    ax = axes[1, i]
    fert_means = []
    fert_names = []
    for fert in fertilizers:
        fert_data = train_engineered[train_engineered['Fertilizer Name'] == fert]
        fert_means.append(fert_data[condition].mean())
        fert_names.append(fert)
    
    bars = ax.bar(range(len(fert_names)), fert_means, color=colors)
    ax.set_xlabel('Fertilizer')
    ax.set_ylabel(f'Average {condition}')
    ax.set_title(f'Average {condition} by Fertilizer')
    ax.set_xticks(range(len(fert_names)))
    ax.set_xticklabels(fert_names, rotation=45)

plt.tight_layout()
plt.show()

# 13. ADVANCED CLUSTERING VISUALIZATION
print("\n13. Advanced Clustering Insights:")

# t-SNE visualization for better cluster separation
from sklearn.manifold import TSNE

print("Computing t-SNE embedding...")
tsne = TSNE(n_components=2, random_state=42, perplexity=50, n_iter=1000)
X_tsne = tsne.fit_transform(X_scaled[:10000])  # Sample for computational efficiency

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# t-SNE by fertilizer
for i, fert in enumerate(fertilizers):
    mask = train_engineered.iloc[:10000]['Fertilizer Name'] == fert
    axes[0].scatter(X_tsne[mask, 0], X_tsne[mask, 1], 
                   c=[colors[i]], label=fert, alpha=0.6, s=1)
axes[0].set_title('t-SNE: Fertilizer Types')
axes[0].legend()

# t-SNE by clusters
cluster_colors = plt.cm.viridis(np.linspace(0, 1, optimal_k))
for cluster_id in range(optimal_k):
    mask = train_clustered.iloc[:10000]['cluster'] == cluster_id
    axes[1].scatter(X_tsne[mask, 0], X_tsne[mask, 1], 
                   c=[cluster_colors[cluster_id]], label=f'Cluster {cluster_id}', alpha=0.6, s=1)
axes[1].set_title('t-SNE: K-Means Clusters')
axes[1].legend()

# Feature importance heatmap by cluster
cluster_feature_importance = []
for cluster_id in range(optimal_k):
    cluster_mask = train_clustered['cluster'] == cluster_id
    cluster_data = X_encoded[cluster_mask]
    cluster_target = target_encoded[cluster_mask]
    
    if len(np.unique(cluster_target)) > 1:  # Only if multiple classes in cluster
        cluster_mi = mutual_info_classif(cluster_data, cluster_target, random_state=42)
        cluster_feature_importance.append(cluster_mi)
    else:
        cluster_feature_importance.append(np.zeros(len(numerical_features)))

cluster_importance_df = pd.DataFrame(cluster_feature_importance, 
                                   columns=numerical_features,
                                   index=[f'Cluster_{i}' for i in range(optimal_k)])

# Plot top features by cluster
top_features_by_cluster = cluster_importance_df.T.nlargest(15, cluster_importance_df.columns[0]).index
sns.heatmap(cluster_importance_df[top_features_by_cluster].T, 
           annot=True, cmap='YlOrRd', fmt='.3f', ax=axes[2])
axes[2].set_title('Feature Importance by Cluster')
axes[2].set_xlabel('Cluster')

plt.tight_layout()
plt.show()


In [ ]:
# 📋 COMPREHENSIVE SUMMARY & RECOMMENDATIONS
print("\n📋 DEEP ANALYSIS SUMMARY & INSIGHTS")
print("=" * 60)

# 14. FEATURE ENGINEERING IMPACT SUMMARY
print("\n14. Feature Engineering Impact Summary:")

# Create comprehensive feature analysis
feature_analysis_summary = pd.DataFrame({
    'Feature_Category': ['Original_Numerical', 'Original_Categorical', 'NPK_Ratios', 
                        'Environmental_Composite', 'Crop_Soil_Interactions', 
                        'NPK_Environment_Interactions', 'Statistical_Interactions'],
    'Count': [6, 2, 6, 4, 4, 6, 6],
    'Avg_MI_Score': [
        np.mean([mi_results[mi_results['Feature'] == f]['Mutual_Information'].iloc[0] 
                for f in ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Phosphorous', 'Potassium']]),
        np.mean([mi_results[mi_results['Feature'] == f]['Mutual_Information'].iloc[0] 
                for f in ['Crop Type', 'Soil Type']]),
        np.mean([mi_results[mi_results['Feature'] == f]['Mutual_Information'].iloc[0] 
                for f in ['N_P_ratio', 'N_K_ratio', 'P_K_ratio', 'NPK_sum', 'NPK_mean', 'NPK_balance'] 
                if f in mi_results['Feature'].values]),
        np.mean([mi_results[mi_results['Feature'] == f]['Mutual_Information'].iloc[0] 
                for f in ['environmental_stress', 'climate_comfort', 'ET_proxy', 'water_availability'] 
                if f in mi_results['Feature'].values]),
        np.mean([mi_results[mi_results['Feature'] == f]['Mutual_Information'].iloc[0] 
                for f in ['crop_water_demand', 'soil_water_retention', 'water_match', 'soil_drainage'] 
                if f in mi_results['Feature'].values]),
        np.mean([mi_results[mi_results['Feature'] == f]['Mutual_Information'].iloc[0] 
                for f in ['N_efficiency', 'P_mobility', 'K_leaching_risk', 'temp_adjusted_N', 'temp_adjusted_P', 'humidity_adjusted_K'] 
                if f in mi_results['Feature'].values]),
        np.mean([mi_results[mi_results['Feature'] == f]['Mutual_Information'].iloc[0] 
                for f in ['temp_humidity', 'moisture_npk', 'temp_nitrogen', 'nitrogen_squared', 'phosphorous_squared', 'potassium_squared'] 
                if f in mi_results['Feature'].values])
    ]
})

print("Feature Category Performance:")
print(feature_analysis_summary.to_string(index=False))

# 15. TOP ACTIONABLE INSIGHTS
print("\n15. Top Actionable Insights:")

insights = [
    "🌱 CROP TYPE is the strongest predictor - Focus on crop-specific fertilizer recommendations",
    "🏔️ SOIL TYPE provides crucial context - Combine with crop type for optimal recommendations", 
    "⚗️ NPK RATIOS are more informative than absolute values - Engineer ratio-based features",
    "🌡️ ENVIRONMENTAL FACTORS have lower individual impact but create meaningful interactions",
    "🔗 CROP-SOIL COMBINATIONS show strong predictive patterns - Create interaction features",
    "📊 CLUSTERING reveals natural fertilizer groups - Use for segmented modeling approaches",
    "🎯 FEATURE INTERACTIONS provide significant value-add over individual features",
    "⚖️ BALANCED CLASSES make this ideal for multi-class classification without sampling concerns"
]

for i, insight in enumerate(insights, 1):
    print(f"{i:2d}. {insight}")

# 16. MODELING RECOMMENDATIONS
print("\n16. Modeling Strategy Recommendations:")

strategies = [
    {
        "Approach": "Baseline Model",
        "Features": "Crop Type + Soil Type only",
        "Expected_Performance": "High (85-90%)",
        "Rationale": "Strong categorical predictors"
    },
    {
        "Approach": "Engineered Feature Model", 
        "Features": "Top 20 engineered features",
        "Expected_Performance": "Very High (90-95%)",
        "Rationale": "Leverages domain knowledge and interactions"
    },
    {
        "Approach": "Ensemble Model",
        "Features": "Multiple feature sets + stacking",
        "Expected_Performance": "Optimal (95%+)",
        "Rationale": "Combines different feature perspectives"
    },
    {
        "Approach": "Rule-Based System",
        "Features": "Decision tree rules",
        "Expected_Performance": "High + Interpretable",
        "Rationale": "Agricultural domain benefits from explainability"
    }
]

strategy_df = pd.DataFrame(strategies)
print(strategy_df.to_string(index=False))

# 17. FEATURE SELECTION RECOMMENDATIONS
print("\n17. Optimal Feature Set Recommendations:")

# Create final feature recommendation
recommended_features = {
    'Core_Categorical': ['Crop Type', 'Soil Type', 'crop_soil_combo'],
    'NPK_Features': ['Nitrogen', 'Phosphorous', 'Potassium', 'N_P_ratio', 'NPK_balance', 'dominant_nutrient'],
    'Environmental': ['Temparature', 'Humidity', 'Moisture', 'water_availability', 'climate_comfort'],
    'Interactions': ['N_efficiency', 'temp_adjusted_N', 'crop_water_demand', 'water_match'],
    'Advanced': ['nitrogen_squared', 'temp_humidity', 'npk_env_ratio']
}

total_recommended = sum(len(features) for features in recommended_features.values())

print(f"Recommended feature set ({total_recommended} features):")
for category, features in recommended_features.items():
    print(f"\n{category} ({len(features)} features):")
    for feature in features:
        if feature in mi_results['Feature'].values:
            mi_score = mi_results[mi_results['Feature'] == feature]['Mutual_Information'].iloc[0]
            print(f"  • {feature} (MI: {mi_score:.4f})")
        else:
            print(f"  • {feature} (Categorical)")

print(f"\n🎯 FINAL RECOMMENDATION:")
print(f"   Use {total_recommended} carefully selected features combining domain knowledge")
print(f"   with data-driven insights for optimal fertilizer recommendation performance.")
print(f"   Focus on crop-soil interactions enhanced with NPK ratios and environmental context.")

# Save engineered dataset
print(f"\n💾 Saving engineered datasets...")
train_engineered.to_csv('datasets/train_engineered.csv', index=False)
test_engineered.to_csv('datasets/test_engineered.csv', index=False)
print(f"✅ Saved train_engineered.csv ({train_engineered.shape}) and test_engineered.csv ({test_engineered.shape})")

print(f"\n🏁 DEEP ANALYSIS COMPLETE!")
print(f"   Original features: {train_df.shape[1] - 1}")  
print(f"   Engineered features: {train_engineered.shape[1] - 1}")
print(f"   Feature improvement: {((train_engineered.shape[1] - train_df.shape[1]) / train_df.shape[1] * 100):.1f}%")
print(f"   Ready for advanced modeling approaches! 🚀")
